In [1]:
from ValueIteration import value_iteration
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools

# Implementing Value Iteration to solve 3-echelon inventory optimisation MDPs

## Centralised system

### No lead times

#### Set up data structures

Notes to self:

Need to decide capacities of warehouse and DC.

In [ ]:
# Function to create state space
def create_state_space(capacity, increment, n_ech=2):
    ''' Creates a set as the state space for an n-echelon problem 
    with format (x1, ..., xn) where xj is the inventory level at site j '''

    # Possible inventory levels at each site
    IL = set(int(x) for x in np.arange(-capacity, capacity+1, increment))

    # Possible set of sets
    S = sorted(set(il_pair for il_pair in itertools.product(IL, repeat=n_ech) if il_pair[1] >= 0))

    # Dictionary containing indices for each state (for value iteration)
    state_idx = {s: i for i, s in enumerate(S)}
    
    return S, state_idx

def create_action_space(capacity, increment, n_ech=2):
    ''' Creates a set as the action space for an n-echelon inventory problem
    with format (q1, ..., qn) where qj is the quantity ordered by site j 
    from site j+1 '''

    # The maximum order quantity at each site is half the capacity of the site
    order_set = set(int(x) for x in np.arange(0, math.ceil(capacity/2)+1, increment))
    
    # Possible actions
    A = sorted(set(order_pair for order_pair in itertools.product(order_set, repeat=n_ech)))

    # Dictionary containing indices for each action (for value iteration)
    action_idx = {a: i for i, a in enumerate(A)}

    return A, action_idx



def create_P_c0(S, A, state_idx, action_idx, demand_distribution, capacity):
    ''' 
    Creates an array containing transition probabilities from s to s' under a
    for a centralised multi-echelon serial system without lead times 
    '''
    def prob_trans(s, a, sp):
        ''' Calculates transition probability from s to s' under action a'''
        prob = 0
        post_order_IL = (s[0] + a[0] , s[1] + a[1] - a[0]) # orders arrive at DC and W, DC order leaves W
        demand = post_order_IL[0] - sp[0]    # retailer demand

        # check that we have a valid next state (truncated if IL exceeds capacity) at the DC
        if (post_order_IL[1] > capacity and sp[1] == capacity) or post_order_IL[1] <= capacity:
            if post_order_IL[0] > capacity: # if on-hand stock at DC exceeds capacity
                if sp[0] < capacity: # not a truncated state - just take demand probability
                    prob = demand_distribution.get(demand, 0)
                elif sp[0] >= capacity: # truncated state - sum relevant demand probabilities
                    prob = sum(demand_distribution[d] for d in demand_distribution if post_order_IL[0] - d >= capacity)
                
            elif post_order_IL[0] - max(demand_distribution) >= -capacity: # if backlogs are not beyond limit
                prob = demand_distribution.get(demand, 0)

            elif post_order_IL[0] - max(demand_distribution) < -capacity: # if backlogs are beyond limit
                if sp[0] > -capacity: # not truncated state - just take demand probability
                    prob = demand_distribution.get(demand, 0)
                elif sp[0] <= -capacity: # truncated state - sum relevant demand probabilities
                    prob = sum(demand_distribution[d] for d in demand_distribution if post_order_IL[0] - d <= -capacity)

        return prob

    # array to store transition probabilities for all combinations of s, a, s'
    P_array = np.zeros((len(S), len(A), len(S)))

    for s in S: # for each state s
        s_idx = state_idx[s]
        for a in A: # for each action a
            a_idx = action_idx[a]
            for sp in S: # for each new state s'
                sp_idx = state_idx[sp]
                # calculate and store probability of transitioning to s' from s under a
                P_array[s_idx, a_idx, sp_idx] = prob_trans(s, a, sp) 
    
    return P_array





        
def create_R_c0(S, A, state_idx, action_idx, demand_distribution, hold_costs, backlog_cost):
    ''' 
    Creates an array containing reward obtained under action a chosen at 
    state s for a centralised multi-echelon serial system without lead times.
    '''

    def cost_function(s, a):
        ''' Calculates the cost incurred if action a is taken at state s. '''
        # Costs at warehouse: holding cost (hc)
        hc_w = hold_costs[1]*max(s[1] + a[1] - a[0], 0)
        warehouse_cost = hc_w

        # Costs at DC: holding cost (hc) and backlog cost (bc)
        hc_dc = hold_costs[0]*sum(max(s[0] + a[0] - dt, 0)*prob for dt, prob in demand_distribution.items())
        bc_dc = backlog_cost*sum(max(dt - s[0] - a[0], 0)*prob for dt, prob in demand_distribution.items())
        dc_cost = hc_dc + bc_dc

        return warehouse_cost + dc_cost
    
    R_array = np.zeros((len(S), len(A)))

    for s in S: # for each state s
        s_idx = state_idx[s]
        for a in A: # for each action a
            a_idx = action_idx[a]
            R_array[s_idx, a_idx] = cost_function(s, a) # calculate reward for taking action a at state s

    return R_array

def c0_value_update_func(state_idx, action_idx, capacity):
    def bellman_eq_2c0(s, S, A, P, R, gamma, Vk):
        ''' Calculates the values from taking each action at state s '''
        s_idx = state_idx[s]
        # ordering decisions should ensure that site capacity is not exceeded and DC orders at most on-hand stock of warehouse
        ########### NEED TO CHECK ALLOWED DECISIONS AND FIX BUG 
        values = dict((a, 0) for a in A if s[0]+a[0] <= capacity and s[1]+a[1] <= capacity and a[0] <= s[1])
        print("s:", s, "possible orders:", values.keys())
        if not values: # if there are no possible ordering decisions, then no units need to be ordered
            values = {(0, 0): 0}
        
        for a in values.keys():
            a_idx = action_idx[a]
            values[a] = R[s_idx, a_idx] + gamma*sum([P[s_idx, a_idx, state_idx[sp]]*Vk[sp] for sp in S])
        return values
    
    return bellman_eq_2c0


In [20]:
# Solving the centralised 3-echelon MDP
capacity = 1000
increment = 100

S, state_idx = create_state_space(capacity, increment)        # state space
A, action_idx = create_action_space(capacity, increment)      # action space
demand_dist = {0: 0.2, 100: 0.6, 200: 0.2}                    # retailer demand distribution
h = [1, 0.5]                                                  # holding cost at DC, W
cb = 100                                                      # backlog penalty cost at DC
P = create_P_c0(S, A, state_idx, action_idx, demand_dist, capacity)
R = create_R_c0(S, A, state_idx, action_idx, demand_dist, hold_costs=h, backlog_cost=cb)
V_init = dict([(s, 0) for s in S])
gamma = 0.999
bellman_eq_c0 = c0_value_update_func(state_idx, action_idx, capacity)

# Run and store results from value iteration
c0_results = value_iteration(S, A, P, R, gamma, max_iterations=50, 
                             bellman_eq=bellman_eq_c0, V_init=V_init, theta=1e-7)
c0_optimal_policy = c0_results["optimal_policy"]
c0_cost_function = c0_results["value_function"]



s: (-1000, 0) possible orders: dict_keys([(0, 0), (0, 100), (0, 200), (0, 300), (0, 400), (0, 500)])
s: (-1000, 100) possible orders: dict_keys([(0, 0), (0, 100), (0, 200), (0, 300), (0, 400), (0, 500), (100, 0), (100, 100), (100, 200), (100, 300), (100, 400), (100, 500)])
s: (-1000, 200) possible orders: dict_keys([(0, 0), (0, 100), (0, 200), (0, 300), (0, 400), (0, 500), (100, 0), (100, 100), (100, 200), (100, 300), (100, 400), (100, 500), (200, 0), (200, 100), (200, 200), (200, 300), (200, 400), (200, 500)])
s: (-1000, 300) possible orders: dict_keys([(0, 0), (0, 100), (0, 200), (0, 300), (0, 400), (0, 500), (100, 0), (100, 100), (100, 200), (100, 300), (100, 400), (100, 500), (200, 0), (200, 100), (200, 200), (200, 300), (200, 400), (200, 500), (300, 0), (300, 100), (300, 200), (300, 300), (300, 400), (300, 500)])
s: (-1000, 400) possible orders: dict_keys([(0, 0), (0, 100), (0, 200), (0, 300), (0, 400), (0, 500), (100, 0), (100, 100), (100, 200), (100, 300), (100, 400), (100, 500)

In [21]:
c0_optimal_policy

{(-1000, 0): (0, 0),
 (-1000, 100): (0, 0),
 (-1000, 200): (0, 0),
 (-1000, 300): (0, 0),
 (-1000, 400): (0, 0),
 (-1000, 500): (0, 0),
 (-1000, 600): (0, 0),
 (-1000, 700): (0, 0),
 (-1000, 800): (0, 0),
 (-1000, 900): (0, 0),
 (-1000, 1000): (0, 0),
 (-900, 0): (0, 0),
 (-900, 100): (100, 0),
 (-900, 200): (100, 0),
 (-900, 300): (100, 0),
 (-900, 400): (100, 0),
 (-900, 500): (100, 0),
 (-900, 600): (100, 0),
 (-900, 700): (100, 0),
 (-900, 800): (100, 0),
 (-900, 900): (100, 0),
 (-900, 1000): (100, 0),
 (-800, 0): (0, 0),
 (-800, 100): (0, 0),
 (-800, 200): (0, 0),
 (-800, 300): (0, 0),
 (-800, 400): (0, 0),
 (-800, 500): (0, 0),
 (-800, 600): (0, 0),
 (-800, 700): (0, 0),
 (-800, 800): (0, 0),
 (-800, 900): (0, 0),
 (-800, 1000): (0, 0),
 (-700, 0): (0, 0),
 (-700, 100): (0, 0),
 (-700, 200): (0, 0),
 (-700, 300): (0, 0),
 (-700, 400): (0, 0),
 (-700, 500): (0, 0),
 (-700, 600): (0, 0),
 (-700, 700): (0, 0),
 (-700, 800): (0, 0),
 (-700, 900): (0, 0),
 (-700, 1000): (0, 0),
 (-60